# 🚗 Dynamic Pricing for Urban Parking Lots
## Models 1, 2, and 3 – Full Implementation (with Explanation)

This notebook implements:
- 📈 **Model 1**: Linear Occupancy-Based Pricing
- 📊 **Model 2**: Demand-Based Pricing (multi-factor)
- 🧭 **Model 3**: Competitive Pricing (geo-aware)

We use the `dataset.csv` file provided in the Summer Analytics 2025 capstone project.


In [1]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, sqrt, atan2


In [2]:
df = pd.read_csv("/content/dataset.csv")
df.head()


ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0      BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1      BHMBCCMKT01       577  26.144536  91.736172         64         car   
2   2      BHMBCCMKT01       577  26.144536  91.736172         80         car   
3   3      BHMBCCMKT01       577  26.144536  91.736172        107         car   
4   4      BHMBCCMKT01       577  26.144536  91.736172        150        bike   

  TrafficConditionNearby  QueueLength  IsSpecialDay LastUpdatedDate  \
0                    low            1             0      04-10-2016   
1                    low            1             0      04-10-2016   
2                    low            2             0      04-10-2016   
3                    low            2             0      04-10-2016   
4                    low            2             0      04-10-2016   

  LastUpdatedTime  
0        07:59:00  
1        08:25:00  
2        08:59:00  
3        09:32:00  
4        09:59:00

In [3]:
# Map vehicle type to weight (bike: 0.5, car: 1.0, truck: 1.5)
vehicle_weights = {'bike': 0.5, 'car': 1.0, 'truck': 1.5}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weights).fillna(1.0)

# Occupancy Rate = Occupancy / Capacity
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']

# Normalize Queue Length
q_min, q_max = df['QueueLength'].min(), df['QueueLength'].max()
df['QueueNorm'] = (df['QueueLength'] - q_min) / (q_max - q_min)

# Map traffic to value and normalize
traffic_map = {'low': 0.2, 'medium': 0.5, 'high': 0.9}
df['TrafficValue'] = df['TrafficConditionNearby'].map(traffic_map).fillna(0.5)

t_min, t_max = df['TrafficValue'].min(), df['TrafficValue'].max()
df['TrafficNorm'] = (df['TrafficValue'] - t_min) / (t_max - t_min)


📈 Model 1 – Baseline Linear Pricing

In [4]:
# Combine date and time columns into one timestamp
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S',
    errors='coerce'
)

# Sort by timestamp just in case
df = df.sort_values(by='Timestamp')


In [5]:
df = df.sort_values(['SystemCodeNumber', 'Timestamp'])

# Initialize price column
df['Price_Model1'] = np.nan

# Set base price
alpha = 5
base_price = 10

# Apply recursively per parking lot
for lot in df['SystemCodeNumber'].unique():
    prev_price = base_price
    for idx in df[df['SystemCodeNumber'] == lot].index:
        occ = df.loc[idx, 'Occupancy']
        cap = df.loc[idx, 'Capacity']
        occ_rate = occ / cap if cap > 0 else 0
        new_price = prev_price + alpha * occ_rate
        df.loc[idx, 'Price_Model1'] = min(max(new_price, 5), 20)
        prev_price = df.loc[idx, 'Price_Model1']


📊 Model 2 – Demand-Based Pricing

In [6]:
# Raw demand function using multiple factors
df['RawDemand'] = (
    df['OccupancyRate'] +
    df['QueueNorm'] -
    df['TrafficNorm'] +
    df['IsSpecialDay'] +
    df['VehicleWeight']
)


In [7]:
# Normalize the demand to range 0–1
df['NormalizedDemand'] = df['RawDemand'] / 5.0  # max theoretical demand score = 5

# Calculate price: base × (1 + demand)
df['Price_Model2'] = base_price * (1 + df['NormalizedDemand'])
df['Price_Model2'] = df['Price_Model2'].clip(lower=5, upper=20)

df[['SystemCodeNumber', 'NormalizedDemand', 'Price_Model2']].head()


SystemCodeNumber  NormalizedDemand  Price_Model2
0      BHMBCCMKT01          0.234477     12.344772
1      BHMBCCMKT01          0.235517     12.355170
2      BHMBCCMKT01          0.254396     12.543963
3      BHMBCCMKT01          0.263755     12.637551
4      BHMBCCMKT01          0.178660     11.786597

🧭 Model 3 – Competitive Pricing (Geo-aware)

In [8]:
df = df.head(500)  # Use only first 5000 rows to validate logic quickly


In [9]:
# Helper function to compute distance between two geo-coordinates
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c  # distance in km


In [10]:
# Create placeholder for Model 3 price
prices_model3 = []

for idx, row in df.iterrows():
    lat1, lon1 = row['Latitude'], row['Longitude']
    this_price = row['Price_Model2']
    nearby_prices = []

    # Compare with all other lots
    for _, other in df.iterrows():
        if row['SystemCodeNumber'] == other['SystemCodeNumber']:
            continue  # skip self
        dist = haversine(lat1, lon1, other['Latitude'], other['Longitude'])
        if dist <= 0.5:  # within 0.5 km
            nearby_prices.append(other['Price_Model2'])

    # Competitive adjustment logic
    if len(nearby_prices) > 0:
        avg_comp_price = np.mean(nearby_prices)

        # If full and own price > competitors → decrease price
        if row['Occupancy'] >= row['Capacity'] and this_price > avg_comp_price:
            this_price *= 0.95

        # If competitors more expensive → increase price
        elif this_price < avg_comp_price:
            this_price *= 1.05

    prices_model3.append(min(max(this_price, 5), 20))  # clip to [5, 20]

df['Price_Model3'] = prices_model3
df[['SystemCodeNumber', 'Price_Model2', 'Price_Model3']].head()


SystemCodeNumber  Price_Model2  Price_Model3
0      BHMBCCMKT01     12.344772     12.344772
1      BHMBCCMKT01     12.355170     12.355170
2      BHMBCCMKT01     12.543963     12.543963
3      BHMBCCMKT01     12.637551     12.637551
4      BHMBCCMKT01     11.786597     11.786597

In [11]:
# Compare all model prices
df[['SystemCodeNumber', 'OccupancyRate', 'Price_Model1', 'Price_Model2', 'Price_Model3']].head(10)


SystemCodeNumber  OccupancyRate  Price_Model1  Price_Model2  Price_Model3
0      BHMBCCMKT01       0.105719     10.528596     12.344772     12.344772
1      BHMBCCMKT01       0.110919     11.083189     12.355170     12.355170
2      BHMBCCMKT01       0.138648     11.776430     12.543963     12.543963
3      BHMBCCMKT01       0.185442     12.703640     12.637551     12.637551
4      BHMBCCMKT01       0.259965     14.003466     11.786597     11.786597
5      BHMBCCMKT01       0.306759     15.537262     13.013518     13.013518
6      BHMBCCMKT01       0.379549     17.435009     12.559099     12.559099
7      BHMBCCMKT01       0.428076     19.575390     12.665676     12.665676
8      BHMBCCMKT01       0.448873     20.000000     12.707271     12.707271
9      BHMBCCMKT01       0.461005     20.000000     10.988677     10.988677

## 📊 Real-Time Visualization with Bokeh

We now visualize pricing behavior using **Bokeh**:

- 📈 Time-series plots for Model 1, 2, and 3
- 🔄 Comparison of price fluctuations for a selected parking lot


In [12]:
# If you're in Colab or need Bokeh for the first time, uncomment and run this:
# !pip install bokeh

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import column
from bokeh.palettes import Category10

output_notebook()


### 🕓 Combine Date and Time

We'll convert the `LastUpdatedDate` and `LastUpdatedTime` columns into a single datetime column to allow proper time-series plotting.


In [13]:
# Convert string date and time into a datetime format
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S',
    errors='coerce'
)

# Fill any missing timestamps just in case
df = df.sort_values('Timestamp')
df['Timestamp'] = df['Timestamp'].fillna(method='bfill')


/tmp/ipython-input-13-2290693505.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Timestamp'] = df['Timestamp'].fillna(method='bfill')


### 📈 Visualization Function

This function generates an interactive Bokeh line chart comparing the prices from Model 1, 2, and 3 for any given parking lot.


In [14]:
def plot_price_models_for_lot(lot_code):
    subset = df[df['SystemCodeNumber'] == lot_code]

    source = ColumnDataSource(data={
        'time': subset['Timestamp'],
        'model1': subset['Price_Model1'],
        'model2': subset['Price_Model2'],
        'model3': subset['Price_Model3']
    })

    p = figure(title=f"Dynamic Pricing Over Time – {lot_code}",
               x_axis_label='Time', y_axis_label='Price ($)',
               x_axis_type='datetime', width=800, height=400)

    p.line(x='time', y='model1', source=source, line_width=2, color=Category10[3][0], legend_label='Model 1')
    p.line(x='time', y='model2', source=source, line_width=2, color=Category10[3][1], legend_label='Model 2')
    p.line(x='time', y='model3', source=source, line_width=2, color=Category10[3][2], legend_label='Model 3')

    hover = HoverTool(
        tooltips=[
            ('Time', '@time{%F %T}'),
            ('Model 1', '@model1'),
            ('Model 2', '@model2'),
            ('Model 3', '@model3')
        ],
        formatters={'@time': 'datetime'},
        mode='vline'
    )
    p.add_tools(hover)
    p.legend.location = 'top_left'
    return p


### 📍 View Plot for a Sample Parking Lot

Use any lot code from the dataset to display a Bokeh line chart.


In [15]:
# Pick a parking lot code (first one by default)
sample_lot = df['SystemCodeNumber'].unique()[0]

# Show interactive Bokeh plot
show(plot_price_models_for_lot(sample_lot))


In [16]:
# Only run this once in Colab or local environment
!pip install pathway


In [17]:
import pathway as pw
import pandas as pd


In [18]:
# Define the input schema for each parking lot entry
class ParkingRecord(pw.Schema):
    timestamp: str
    parking_lot_id: str
    latitude: float
    longitude: float
    capacity: int
    occupancy: int
    queue_length: int
    vehicle_type: str
    traffic: float
    is_special_day: int


In [19]:
@pw.udf
def compute_price(occupancy, capacity, queue_length, traffic, is_special_day, vehicle_type):
    occ_rate = occupancy / capacity if capacity > 0 else 0
    queue_norm = min(queue_length / 10, 1)
    traffic_norm = min(traffic / 10, 1)
    vehicle_weights = {'bike': 0.5, 'car': 1.0, 'truck': 1.5}
    vehicle_weight = vehicle_weights.get(vehicle_type.lower(), 1.0)

    # Demand Calculation
    demand = occ_rate + queue_norm - traffic_norm + is_special_day + vehicle_weight
    demand = max(0, min(demand / 5, 1))  # Normalize

    base_price = 10
    price = base_price * (1 + demand)
    return round(min(max(price, 5), 20), 2)


In [20]:
# Stream input CSV (in real-time simulation mode)
input_table = pw.io.csv.read(
    'dataset_stream.csv',   # Make sure this CSV is preprocessed
    schema=ParkingRecord,
    mode='streaming'  # Key for simulating real-time flow
)


In [21]:
# Updated select() call with explicit field unpacking
output_table = input_table.select(
    timestamp = pw.this.timestamp,
    parking_lot_id = pw.this.parking_lot_id,
    price = compute_price(
        pw.this.occupancy,
        pw.this.capacity,
        pw.this.queue_length,
        pw.this.traffic,
        pw.this.is_special_day,
        pw.this.vehicle_type
    )
)


In [22]:
# Launch the real-time simulation
pw.run()


Output()

In [23]:
# Optional: Convert your original CSV into Pathway format
df = pd.read_csv("dataset.csv")

df_out = pd.DataFrame({
    "timestamp": pd.to_datetime(df["LastUpdatedDate"] + ' ' + df["LastUpdatedTime"],
                                format="%d-%m-%Y %H:%M:%S", errors='coerce'),
    "parking_lot_id": df["SystemCodeNumber"],
    "latitude": df["Latitude"],
    "longitude": df["Longitude"],
    "capacity": df["Capacity"],
    "occupancy": df["Occupancy"],
    "queue_length": df["QueueLength"],
    "vehicle_type": df["VehicleType"],
    "traffic": df["TrafficConditionNearby"].map({"low": 2, "medium": 5, "high": 9}),
    "is_special_day": df["IsSpecialDay"]
})

df_out = df_out.dropna()
df_out.to_csv("dataset_stream.csv", index=False)
